# B. Команда аналитиков (задача C/29448) (диагностика технических навыков - аналитика)
* Ограничение времени	1 секунда
* Ограничение памяти	256Mb
* Ввод	стандартный ввод или input.txt
* Вывод	стандартный вывод или output.txt

## Условие:
* Аркадий — менеджер команды аналитиков из $k$ человек (*мы считаем всех членов команды равнозначными*). 
* У Аркадия есть бэклог из $n$ задач, $i$-я задача требует $t_i$ дней работы *любого* из членов команды . 
* Над каждой задачей от начала до конца должен работать кто-то один — передавать задачи в процессе выполнения неудобно. 
* Для каждой задачи известен дедлайн — $d_i$ рабочих дней.

## Найти:
Помогите Аркадию определить, **успеет ли его команда выполнить все задачи в срок**.

## Формат ввода:
* В первой строке заданы два целых положительных числа — $n$ и $k$ ($1 \le n$, $k \le 15$). 
* В каждой из следующих $n$ строк заданы два целых положительных числа — $t_i$ и $d_i$ ($1 \le t_i \le d_i \le 109$).

## Формат вывода:
* Выведите `NO`, если нельзя выполнить все задачи в срок.

* Иначе в первой строке выведите `YES`. 
* Далее выведите $k$ строк, причем в $i$-й строке будет описание того, какие задачи выполняет $i$-й сотрудник: 
 * сначала выведите одно целое неотрицательное число $T_i$ — количество задач, которое будет выполнять $i$-й сотрудник, 
 * а зачем выведите $T_i$ чисел — номера задач, которые будет выполнять i-й сотрудник. 
 * (Выводите номера задач в том порядке, в котором их должен выполнять сотрудник. Задачи нумеруются в порядке перечисления во входных данных.)

Если существует несколько правильных ответов, вы можете вывести любой.

In [1]:
import pandas as pd
#--------------------------------
# arrange the input of given data

tmp_str = input()
n = int(tmp_str.split()[0])
k = int(tmp_str.split()[1])

if (1 <= n <= 15) and (1 <= k <= 15):
    nk_ok = True
else:
    nk_ok = False

if nk_ok:
    tasks = []
    for i in range(n):
        tmp_str = input()
        no = i+1                     # keep initial tasks numbering as task_id
        t = int(tmp_str.split()[0])
        d = int(tmp_str.split()[1])
        s = d - t                    # last available day for starting the task
        f = False                    # assigned status flag
        tasks.append((no,t,d,s,f))

        tasks_df = pd.DataFrame( 
            tasks,
            columns = ['task_id', 'time_todo', 'deadline', 'latest_start', 'assigned']
        )

    if ( (tasks_df['time_todo'] < 1).sum() + (tasks_df['time_todo'] > tasks_df['deadline']).sum()
         + (tasks_df['deadline'] > 109).sum() ) == 0:
        input_ok = True
    else:
        input_ok = False
else:
    input_ok = False


#----------------------------------------------------------------------------------------------------------
# Solution idea:
# let's consider all possible start days for each task and see the max amount of active tasks over all days
# if this max amount of active tasks is bigger in some day than k - this variant is bad

# ----------------------------------------------------------------------------
# first build a list of possible combinations of starting days for all n tasks

variants_list = []

for tidx in range(n):
    
    if not tidx:
        for i in range(tasks_df.loc[tidx,'latest_start']+1):
            variants_list.append([i])
    else:
        new_list = []
        for i in range(tasks_df.loc[tidx,'latest_start']+1):
            for j in range(len(variants_list)):
                new_list.append(variants_list[j]+[i])
        variants_list = new_list


#-------------------------------------------------------------------------------------
# now for each variant calculate how many tasks are maximally active over all 109 days

found = False
last_day = tasks_df['deadline'].max()

for variant in variants_list:
    
    if found:
        break
    
    tasks_df['start'] = variant
    max_tasks = 0
    
    for day in range(last_day+1):
        day_tasks = ( ( tasks_df['start'] <= day ) & ( (tasks_df['start'] + tasks_df['time_todo']) > day) ).sum()
        max_tasks = max(max_tasks,day_tasks)
        
    if max_tasks <= k:
            found = True

if not found:
    print('NO')
else:
    print('YES')

#------------------------------------------------------
# now we know in which order we should assign the tasks

    tasks_df = tasks_df.sort_values('start').reset_index(drop=True)
    
#--------------------------------------------
# create the roster board for the programmers

    guys_df = pd.DataFrame(
                              index = list(range(k)),
                              columns = ['tasks_list','available_since']
                          )
    guys_df['tasks_list'] = [ [] for i in range(k)]
    guys_df['available_since'] = [0 for i in range(k)]

#---------------------------------------------------------------------------------------------------------
# now we run through the scrum-sprint assigning the tasks in the preference order to available programmers

    for day in range(last_day+1):
    
        if (guys_df['available_since'] <= day).sum() == 0:
            continue
    
        for man in range(k):
            if guys_df.loc[man,'available_since'] <= day:
                for tidx in range(n):
                    if tasks_df.loc[tidx,'assigned']:
                        continue
                    tasks_df.loc[tidx,'assigned'] = True
                    guys_df.loc[man,'available_since'] = day + tasks_df.loc[tidx,'time_todo']
                    guys_df.loc[man,'tasks_list'].append(tasks_df.loc[tidx,'task_id'])
                    break

    for l in range(k):
        print( len(guys_df.loc[l,'tasks_list']), end='')
        for t in range( len(guys_df.loc[l,'tasks_list']) ):
            print( guys_df.loc[l,'tasks_list'][t], end='' )
        print()

1 2
1 2
YES
11
0
